In [1]:
%pwd

'd:\\ml-project\\Wine-Quality-Project\\research'

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\ml-project\\Wine-Quality-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path   
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [6]:
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories, save_json

# Configuration for model evaluation
class ConfiguratioManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name
        )
        
        return model_evaluation_config

In [7]:
import os
import pandas as pd
from src.mlproject import logger
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from urllib.parse import urlparse
import joblib

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        """Compute RMSE, MAE, and R2 score."""
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    

    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)

        (rmse, mae , r2) = self.eval_metrics(test_y, predicted_qualities)

        # Save metrics
        scores = {'rmse': rmse,'mae': mae,'r2': r2}

        save_json(path=Path(self.config.metric_file_name), data=scores)


In [8]:
try:
    config = ConfiguratioManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)  
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2025-04-02 17:51:06,282: INFO: common: YAML file config\config.yaml loaded successfully.]
[2025-04-02 17:51:06,286: INFO: common: YAML file params.yaml loaded successfully.]
[2025-04-02 17:51:06,290: INFO: common: YAML file schema.yaml loaded successfully.]
[2025-04-02 17:51:06,293: INFO: common: created directory at : artifacts]
[2025-04-02 17:51:06,296: INFO: common: created directory at : artifacts/model_evaluation]
[2025-04-02 17:51:06,421: INFO: common: JSON file saved successfully at: artifacts\model_evaluation\metrics.json]
